TODO:
Embed all the articles and save the embeddings next to the article text.



Details:
Each of the 6 sections has a short heading and a bit longer title : 6 section (title+heading) embeddings.

Each section has multiple subsections. Each subsection has a heading: one embedding for each subsection (section title + section heading + subsection heading and all article titles in this subsection)

Each subsection has multiple articles. Each article has a title, content, link and links to related articles.

One embedding for each article. Include section title + section heading + subsection heading + article title + article content. (NB article title and section heading is already added to markdown
file.)




In [3]:
import os

import openai
# openai.api_key = ''

In [6]:
EMBEDDING_MODEL = "text-embedding-ada-002"


def get_embedding(text: str, model: str=EMBEDDING_MODEL) -> list[float]:
    result = openai.Embedding.create(
        model=model,
        input=text
    )
    return result["data"][0]["embedding"]

In [17]:
import json
import numpy as np
with open("scraped-data/index.json", "r") as f:
    data = json.load(f)

In [25]:
section_title_heading_embeddings = []
for section in data:
    section_title_heading_embeddings.append(get_embedding(section["title"] + section["heading"]))

with open('embeddings/section_title_heading_embeddings.npy', 'wb') as f:
    np.save(f, np.array(section_title_heading_embeddings))

In [31]:
import os

for section in data:
    subsection_embeddings = []
    for subsection in section["subsections"]:
        article_titles = [article["title"] for article in subsection["articles"]]
        items = [section["title"], section["heading"], subsection["heading"], '\n'.join(article_titles)]
        content = '\n\n'.join(items)
        subsection_embeddings.append(get_embedding(content))
        folder_path = 'embeddings/' + '/'.join(subsection['folder_path'].split('/')[1:])
        os.makedirs(folder_path, exist_ok=True)
        with open(folder_path + '/subsection_embeddings.npy', 'wb') as f:
            np.save(f, np.array(subsection_embeddings))

In [33]:
for section in data:
    for subsection in section["subsections"]:
        folder_path = 'embeddings/' + '/'.join(subsection['folder_path'].split('/')[1:])
        os.makedirs(folder_path, exist_ok=True)

        article_embeddings = []
        for article in subsection["articles"]:
            with open(article['folder_path'] + '/content.md', 'r') as f:
                article_md = f.read()
            items = [section["title"], section["heading"], article_md]
            content = '\n\n'.join(items)
            article_embedding = get_embedding(content)
            article_embeddings.append(article_embedding)
            with open(folder_path + '/article_embeddings.npy', 'wb') as f:
                np.save(f, np.array(article_embeddings))